In [41]:
import wget
import os
import re
import requests
import glob
import io

from datetime import datetime
from bs4 import BeautifulSoup

In [42]:
url = 'https://presidente.gob.mx/16-02-23-version-estenografica-inauguracion-de-vialidad-principal-al-aifa-camino-tonanitla-desde-nextlalpan-estado-de-mexico/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
response = requests.get(url, headers=headers)
with open('conferencia.txt', 'wb') as f:
    f.write(response.content)

In [43]:
url = 'https://presidente.gob.mx/secciones/version-estenografica/page/2/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a', href=True)
links = [link['href'] for link in links if re.search(r'16-02-23', link['href'])]
for link in links:
    response = requests.get(link, headers=headers)
    with open('conferencia.txt', 'ab') as f:
        f.write(response.content)

In [44]:
reg = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'
re.findall(
    reg,
    open('./conferencia.txt', 'r', encoding='utf-8').read())[0]

('http',
 '151.394.mwp.accessdomain.com',
 '/wp-content/themes/diplomat/favicon.ico')

In [17]:
url_page = 'https://presidente.gob.mx/secciones/version-estenografica/page/'
n_pag = 117

if not os.path.exists('./pages'):
    os.makedirs('./pages')

for i in range(n_pag):
    response = requests.get(url_page + str(i+1) + '/', headers=headers)
    with open('./pages/' + str(i+1) + '.txt', 'wb') as f:
        f.write(response.content)

In [45]:
urls_conf = []

for f_pag in glob.glob('./pages/*.txt'):
    urls_conf += [ path for protocol, domain, path in re.findall(
        reg,
            open(f_pag, 'r', encoding='utf-8').read()
        )]

urls_conf, len(set(urls_conf))
    

(['/wp-content/themes/diplomat/favicon.ico',
  '/feed/',
  '/xmlrpc.php',
  '',
  '/wp-content/themes/diplomat/helper/capcha/image.php/',
  '/wp-content/themes/diplomat/',
  '/wp-admin/admin-ajax.php',
  '/feed/',
  '/comments/feed/',
  '/secciones/version-estenografica/feed/',
  '/wp-content/plugins/arqam/assets/style.css?ver=e283627719625c42dbfcc13b6adb6f19',
  '/css?family=Roboto+Slab%7CRoboto%7CPT+Serif%7CDroid+Serif%7CMontserrat&#038',
  '/wp-content/plugins/tmm_content_composer/css/fontello.css?ver=e283627719625c42dbfcc13b6adb6f19',
  '/wp-content/plugins/LayerSlider/assets/static/layerslider/css/layerslider.css?ver=7.2.1',
  '/wp-includes/css/dist/block-library/style.min.css?ver=e283627719625c42dbfcc13b6adb6f19',
  '/wp-content/plugins/foobox-image-lightbox/free/css/foobox.free.min.css?ver=2.7.17',
  '/wp-content/plugins/tmm_events_calendar/css/styles.css?ver=e283627719625c42dbfcc13b6adb6f19',
  '/wp-content/plugins/tmm_events_calendar/css/calendar.css?ver=e283627719625c42dbfcc1

In [54]:
urls_conf_set = list(set(urls_conf))
print(len(urls_conf_set))

if not os.path.exists('./estenograficas'):
    os.makedirs('./estenograficas')

for path in urls_conf_set:
    url_path = 'https://presidente.gob.mx' + path
    response = requests.get(url_path, headers=headers)
    if os.path.exists('./estenograficas/' + path.replace('/','.') + '.txt'):
        with open('./estenograficas/' + path.replace('/','-') + '.txt', 'w', encoding='utf-8') as f:
            f.write(response.text)

3642


KeyboardInterrupt: 

In [55]:
if not os.path.exists('./estenograficas_limpias'):
    os.makedirs('./estenograficas_limpias')

for f_page in glob.glob('./estenograficas/*.txt'):
    soup = BeautifulSoup(open(f_page, 'r', encoding='utf-8').read(), 'html.parser')
    open('./estenograficas_limpias' + f_page.replace(
        './estenograficas', ''), 'w', encoding='utf-8').write(soup.get_text())
    

In [56]:
def normaliza(fecha):
    datetimeobject = datetime.strptime(fecha[:6] + "20" + fecha[6:], '%d.%m.%Y')
    newformat = datetimeobject.strftime('%Y-%m-%d')
    return newformat

In [63]:
i = 0

if not os.path.exists('./estenograficas_limpias_por_fecha'):
    os.makedirs('./estenograficas_limpias_por_fecha')

fechas_conferencia = []

for f_page in glob.glob('./estenograficas_limpias/*.txt'):
    texto = open(f_page, 'r', encoding='utf-8').read()
    fechas_en_archivo = re.findall(
        '[0-9][0-9]\.[0-9][0-9]\.[0-9][0-9]',
        texto)
    print(fechas_en_archivo)
    if len(fechas_en_archivo) == 4:
        if not os.path.isfile('./estenograficas_limpias_por_fecha/' + fechas_en_archivo[0].replace('.','-')):
            open('./estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0]), 'w', encoding='utf-8').write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
        elif not os.path.isfile('./estenograficas_limpias_por_fecha/' + fechas_en_archivo[0].replace('.','-') + '_2da'):
            open('./estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0]) + '_2da', 'w', encoding='utf-8').write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
        elif not os.path.isfile('./estenograficas_limpias_por_fecha/' + fechas_en_archivo[0].replace('.','-') + '_3ra'):
            open('./estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0]) + '_3ra', 'w', encoding='utf-8').write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]

['03.12.21', '03.12.21', '03.12.21', '03.12.21']
['06.12.19', '06.12.19', '06.12.19', '06.12.19']
['09.04.22', '09.04.22', '09.04.22', '09.04.22']
['09.12.22', '09.12.22', '09.12.22', '09.12.22']
['11.02.23', '11.02.23', '11.02.23', '11.02.23']
['12.06.21', '12.06.21', '12.06.21', '12.06.21']
['14.01.23', '14.01.23', '14.01.23', '14.01.23']
['20.01.21', '20.01.21', '20.01.21', '20.01.21']
['24.11.22', '24.11.22', '24.11.22', '24.11.22']
['28.04.22', '28.04.22', '28.04.22', '28.04.22']
['28.12.21', '28.12.21', '28.12.21', '28.12.21']
[]
['14.03.19', '14.03.19', '14.03.19', '14.03.19']
['12.05.19', '12.05.19', '12.05.19', '12.05.19']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
